# Manifold GP Supervised Learning via Precision Matrix

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
from mayavi import mlab
from importlib.resources import files
from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from manifold_gp.utils.generate_truth import groundtruth_from_mesh

Select the device: GPU or CPU.

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Load Dataset and Generate Ground Truth

In [3]:
data_path = files('manifold_gp.data').joinpath('dragon10k.stl')
nodes, faces, truth = groundtruth_from_mesh(data_path)
sampled_x = torch.from_numpy(nodes).float().to(device)
sampled_y = torch.from_numpy(truth).float().to(device)
(m, n) = sampled_x.shape

## Apply Noise to the Dataset

In order to simulate real case scenario we can apply a normally distributed noise to our dataset. Here we can consider to apply the noise either to the sampled data points from the manifold or to the ground truth relative to each labeled points.

### Apply noise to the manifold
Setting the variable `manifold_noise` you can apply Gaussian noise to the sampled points from the manifold.

In [4]:
manifold_noise = 0.0
noisy_x = sampled_x + manifold_noise * torch.randn(m, n).to(device)

### Apply noise to the ground truth
Setting the variable `function_noise` you can apply Gaussian noise to the training points.

In [5]:
function_noise = 0.0
noisy_y = sampled_y + function_noise * torch.randn(m).to(device)

## Training Dataset

Create the training dataset. You can set the variable `num_train` to define the number of points within your dataset.

In [6]:
num_train = 500
train_idx = torch.randperm(m)[:num_train]
train_x = noisy_x[train_idx, :]
train_y = noisy_y[train_idx]

## Test Dataset

Create the training dataset. You can set the variable `num_test` to define the number of points within your dataset.

In [7]:
num_test = 100
test_idx = torch.randperm(m)[:num_test]
test_x = noisy_x[test_idx, :]
test_y = noisy_y[test_idx]

## Initialize Kernel

Create the Riemann Matern kernel. `nu` control the kernel smoothness like in the stadard Matern kernel. `neighbors` set the number of nearest neighbors in the KNN-based discrete approximation of the Laplace-Beltrami operator and construction of differentiable eigenfunctions. `modes` set the number of the eigenfunctions used to approximate the Riemann Matern kernel. As for classical *GPyTorch* kernels the Riemann Matern kernel can be decorated using with the *ScaleKernel* to have the signal variance parameter.

In [8]:
nu = 2
neighbors = 50
modes = 20
kernel = gpytorch.kernels.ScaleKernel(RiemannMaternKernel(noisy_x, nu, neighbors, modes))

## Initialize Likelihood and GP Model

Define the type of likelihood and the GP model. The GP model takes as input the training set and and labels, the likelihood and the Riemann Matern Kernel.

In [9]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = RiemannGP(noisy_x, noisy_y, likelihood, kernel).to(device)

## Train Model

Train the GP model using the fast Lanczos precision matrix optimization. `lr` set the learning rate; `iters` defines the number of iteratios; `verbose` can be set to `True` for printing the parameters each iteration.

In [10]:
lr = 1e-2
iters = 100
verbose = True
model.manifold_informed_train(lr, iters, verbose)

Iteration: 0, Loss: 1839905.875, Noise Variance: 0.007, Signal Variance: 1.000, Lengthscale: 0.368, Epsilon: 0.135
Iteration: 1, Loss: 1758323.750, Noise Variance: 0.007, Signal Variance: 1.006, Lengthscale: 0.371, Epsilon: 0.137
Iteration: 2, Loss: 1680530.750, Noise Variance: 0.007, Signal Variance: 1.013, Lengthscale: 0.374, Epsilon: 0.138
Iteration: 3, Loss: 1606415.250, Noise Variance: 0.007, Signal Variance: 1.019, Lengthscale: 0.377, Epsilon: 0.139
Iteration: 4, Loss: 1535861.000, Noise Variance: 0.007, Signal Variance: 1.025, Lengthscale: 0.380, Epsilon: 0.140
Iteration: 5, Loss: 1468746.750, Noise Variance: 0.007, Signal Variance: 1.032, Lengthscale: 0.383, Epsilon: 0.142
Iteration: 6, Loss: 1404949.625, Noise Variance: 0.007, Signal Variance: 1.038, Lengthscale: 0.387, Epsilon: 0.143
Iteration: 7, Loss: 1344342.625, Noise Variance: 0.007, Signal Variance: 1.044, Lengthscale: 0.390, Epsilon: 0.144
Iteration: 8, Loss: 1286799.875, Noise Variance: 0.007, Signal Variance: 1.051, 

Iteration: 72, Loss: 226763.781, Noise Variance: 0.010, Signal Variance: 1.320, Lengthscale: 0.539, Epsilon: 0.208
Iteration: 73, Loss: 223323.312, Noise Variance: 0.010, Signal Variance: 1.323, Lengthscale: 0.540, Epsilon: 0.209
Iteration: 74, Loss: 219974.484, Noise Variance: 0.010, Signal Variance: 1.325, Lengthscale: 0.542, Epsilon: 0.210
Iteration: 75, Loss: 216713.828, Noise Variance: 0.010, Signal Variance: 1.328, Lengthscale: 0.543, Epsilon: 0.210
Iteration: 76, Loss: 213538.219, Noise Variance: 0.010, Signal Variance: 1.330, Lengthscale: 0.545, Epsilon: 0.211
Iteration: 77, Loss: 210445.438, Noise Variance: 0.010, Signal Variance: 1.332, Lengthscale: 0.546, Epsilon: 0.212
Iteration: 78, Loss: 207430.797, Noise Variance: 0.010, Signal Variance: 1.335, Lengthscale: 0.547, Epsilon: 0.212
Iteration: 79, Loss: 204490.656, Noise Variance: 0.010, Signal Variance: 1.337, Lengthscale: 0.549, Epsilon: 0.213
Iteration: 80, Loss: 201625.031, Noise Variance: 0.010, Signal Variance: 1.340, 

tensor(158047.5156, device='cuda:0', grad_fn=<MulBackward0>)

## Model Evaluation

In [11]:
%%capture
likelihood.eval()
model.eval()

Getting faster predictive distributions using [LOVE](https://arxiv.org/abs/1803.06058). We compute the **mean**, the **standard deviation** and one **posterior sample**. In addition we evaluate the kernel at the first point of our dataset.

In [12]:
%%capture
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds = likelihood(model(sampled_x))
    mean = preds.mean
    std = preds.stddev
    posterior_sample = preds.sample()
    kernel_eval = kernel(sampled_x[0, :].unsqueeze(0), sampled_x).evaluate().squeeze()

## Plot Results

In [13]:
with torch.no_grad():
    # Bring data to cpu
    if use_cuda:
        sampled_x = sampled_x.cpu().numpy()
        sampled_y = sampled_y.cpu().numpy()
        train_x = train_x.cpu().numpy()
        kernel_eval = kernel_eval.cpu().numpy()
        posterior_sample = posterior_sample.cpu().numpy()
        mean = mean.cpu().numpy()
        std = std.cpu().numpy()

In [14]:
%%capture
%%bash
jupyter nbextension install --py mayavi --user

In [15]:
%%capture
mlab.init_notebook()
v_options = {'mode': 'sphere','scale_factor': 3e-3, 'color': (0, 0, 0)}

### Ground Truth with Sample Training Points

In [16]:
mlab.figure(fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=sampled_y)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### Mean

In [17]:
mlab.figure(fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=mean)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### Standard Deviation

In [18]:
mlab.figure(fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=std)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### One Posterior Sample

In [19]:
mlab.figure(fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=posterior_sample)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### Kernel Evaluation

In [20]:
mlab.figure(fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=kernel_eval)
mlab.colorbar(orientation='vertical')
mlab.points3d(sampled_x[0,0], sampled_x[0,1], sampled_x[0,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…